# 👋🌍 Hello, world: Submit a Pulser job to Pasqal

In this notebook, we'll review the basics of Azure Quantum by submitting a simple *job*, or quantum program, to [Pasqal](https://pasqal.com/). We will use [Pulser](https://pulser.readthedocs.io/) to define a quantum job that can run on Pasqal's hardware.

## Submit a simple job to Pasqal using Azure Quantum
Azure Quantum provides several ways to express quantum programs. In this example we are using Pulser, the quantum computing library of Pasqal to program arrays of neutral atoms. All code in this example will be written in Python.

Let's begin. When you see a code block, hover over it and click the triangle play-button to execute it. To avoid any compilation issues, this should be done in order from top to bottom.

### 1. Connect to the Azure Quantum workspace


In [ ]:
from azure.quantum import Workspace

# Your `resource_id` should be available on the Overview page of your Quantum Workspace.

workspace = Workspace(resource_id = "")

Let's see whether the Pasqal provider is enabled in this workspace with the following command:


In [ ]:
print("This workspace's targets:")
for target in workspace.get_targets():
    print("-", target.name)

### ❕ Do you see pasqal.sim.emu-tn in your list of targets? If so, you're ready to keep going.

Don't see it? You may need to add Pasqal to your workspace to run this sample. Navigate to the **Providers** page in the portal and click **+Add** to add the Pasqal provider.

## Pasqal: The quantum provider
Azure Quantum partners with third-party companies to deliver solutions to quantum jobs. These company offerings are called *providers*. Each provider can offer multiple *targets* with different capabilities. See the table below for Pasqal's targets.

| Target name | Target ID            | Number of qubits              | Description                                                                                                                          |
|-------------|----------------------|-------------------------------|--------------------------------------------------------------------------------------------------------------------------------------|
| Emu-TN      | `pasqal.sim.emu-tn`  | 100 qubits 1D and 2D networks | Simulates the time-evolution of a quantum state using the Schrödinger equation corresponding to the actions that the lasers perform. |
| Fresnel1    | `pasqal.qpu.fresnel` | 100 qubits                    | PASQAL's neutral atoms quantum computer.                                                                                             |

For this example, we will use `pasqal.sim.emu-tn`. To learn more about Pasqal's targets, check out our [documentation](https://learn.microsoft.com/azure/quantum/provider-pasqal).

### 2. Build the quantum program

Let's create a simple Pulser Sequence to run on an array of neutral atoms (a Register).

In [ ]:
import math
import pulser

reg = pulser.Register.rectangle(1, 2, spacing=8, prefix="atom")
reg.draw()

duration = 1000  # Typical: ~1 µsec
pi_pulse = pulser.Pulse.ConstantDetuning(
    pulser.BlackmanWaveform(duration, math.pi), 0.0, 0.0
)

seq = pulser.Sequence(reg, pulser.DigitalAnalogDevice)

seq.declare_channel("ryd", "rydberg_local", "atom0")

seq.add(pi_pulse, "ryd")
seq.target("atom1", "ryd")
seq.add(pi_pulse, "ryd")
seq.measure()

seq.draw()

Each pulse acts on a set of atoms at a certain moment of time. The entire `Sequence` is stored by Pulser and can then be either simulated or sent to a real device.

We've just built a sequence sending the same π-pulse to two atoms, sequentially, using the same channel. With Pasqal's idealized simulator, we will be able to simulate the quantum evolution of both atoms over time.

### 3. Submit the quantum program to Pasqal

In [ ]:
# Using the Pasqal simulator target, call "run" to submit the job. We'll
# use 100 repetitions (simulated runs).
target = workspace.get_targets(name="pasqal.sim.emu-tn")
job = target.submit(
    input_data=dict(sequence_builder=seq.to_abstract_repr()),
    shots=100,
    input_data_format="pasqal.pulser.v1", 
    output_data_format="pasqal.pulser-results.v1",
    name="Pasqal sequence",
)

# Print the job ID.
print("Job id:", job.id)

The job ID can be used to retrieve the results later using the [get_job method](https://learn.microsoft.com/python/azure-quantum/azure.quantum.workspace?#azure-quantum-workspace-get-job) or by viewing it under the **Job management** section of the portal.

### 4. Obtain the job results
Let's await the job execution by calling `job.get_results()`. This may take a minute or so ⏳. Your job is being packaged and sent to Pasqal, where it will wait its turn to be run.

In [ ]:
# Await job results.
print("Awaiting job results...")
result = job.get_results()

# To view each measured qubit state, you can print the result.
print("Job finished. Measure count for each state:")
print(result)

**See the result above? Congratulations, you've submitted a Pasqal job with Azure Quantum! 👏**

### 5. Next steps
Next, you can try running a program on Pasqal's hardware target. Just replace `pasqal.sim.emu-tn` with `pasqal.qpu.fresnel`. Don't worry - your work here is automatically saved.

To learn more about submitting Pulser sequences to Azure Quantum, review [this documentation](https://learn.microsoft.com/en-us/azure/quantum/quickstart-microsoft-provider-format?tabs=tabid-portal%2Ctabid-pyquil#submit-a-circuit-to-pasqal-using-pulser-sdk).

To learn more about job pricing, review the [Azure Quantum documentation on job costs](https://learn.microsoft.com/en-us/azure/quantum/pricing#pasqal).